In [1]:
import numpy as np
import pandas as pd

# Task 0
Read the dataset from csv file & perform data cleaning - remove all rows, which contains `?` in some columns.
Also check for data correctness (salary & salary $K).

In [49]:
data = pd.read_csv(
    "/Users/rurakite/PycharmProjects/py-adult-data-analysis/data/adult.csv"
)
cleaned_data = data[~data.isin(["?"]).any(axis=1)]

salary_correct = (
    ((cleaned_data["salary"] == "<=50K") & (cleaned_data["salary K$"] <= 50))
    | ((cleaned_data["salary"] == ">50K") & (cleaned_data["salary K$"] > 50))
).all()


print("\nData Correctness:")
print(f"All 'salary' entries are correct: {salary_correct}")


Data Correctness:
All 'salary' entries are correct: True


,Unnamed: 0,age,workclass,education,marital-status,occupation,relationship,race,sex,hours-per-week,native-country,salary,salary K$
0,0,39,State-gov,Bachelors,Never-married,Adm-clerical,Not-in-family,White,Male,40,United-States,<=50K,39
1,1,50,Self-emp-not-inc,Bachelors,Married-civ-spouse,Exec-managerial,Husband,White,Male,13,United-States,<=50K,35
2,2,38,Private,HS-grad,Divorced,Handlers-cleaners,Not-in-family,White,Male,40,United-States,<=50K,27
3,3,53,Private,11th,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,40,United-States,<=50K,43
4,4,28,Private,Bachelors,Married-civ-spouse,Prof-specialty,Wife,Black,Female,40,Cuba,<=50K,25
...,...,...,...,...,...,...,...,...,...,...,...,...,...
32556,32556,27,Private,Assoc-acdm,Married-civ-spouse,Tech-support,Wife,White,Female,38,United-States,<=50K,36
32557,32557,40,Private,HS-grad,Married-civ-spouse,Machine-op-inspct,Husband,White,Male,40,United-States,>50K,173
32558,32558,58,Private,HS-grad,Widowed,Adm-clerical,Unmarried,White,Female,40,United-States,<=50K,40
32559,32559,22,Private,HS-grad,Never-married,Adm-clerical,Own-child,White,Male,20,United-States,<=50K,38


# Task 1
Print the count of men and women in the dataset.

In [46]:
df = pd.DataFrame(cleaned_data)

men_count = df[df["sex"] == "Male"].shape[0]
women_count = df[df["sex"] == "Female"].shape[0]

print(f"Men count: {men_count}")
print(f"Women count: {women_count}")

Men count: 20380
Women count: 9782


# Task 2
Find the average age of men in dataset

In [57]:
average_age_men = round(df[df["sex"] == "Male"]["age"].mean())

print(f"Average age of men: {average_age_men}")

Average age of men: 39


# Task 3
Get the percentage of people from Poland (native-country)

In [52]:
poland_count = df[df["native-country"] == "Poland"].shape[0]
poland_percentage = round(((poland_count / 32560) * 100), 2)

print(f"Percentage of people from Poland: {poland_percentage}%")

Percentage of people from Poland: 0.17%


# Task 4
Get the mean and standard deviation of the age for people who earn > 50K per year. After this, get it for those who earn <= 50K.

In [53]:
mean_age_high_income = round(df[df["salary"] == ">50K"]["age"].mean(), 2)
std_age_high_income = round(df[df["salary"] == ">50K"]["age"].std(), 2)

mean_age_low_income = round(df[df["salary"] == "<=50K"]["age"].mean(), 2)
std_age_low_income = round(df[df["salary"] == "<=50K"]["age"].std(), 2)



print(f"People earning >50K:")
print(f"Mean age: {mean_age_high_income}")
print(f"Standard deviation of age: {std_age_high_income}")

print(f"\nPeople earning <=50K:")
print(f"Mean age: {mean_age_low_income}")
print(f"Standard deviation of age: {std_age_low_income}")

People earning >50K:
Mean age: 43.96
Standard deviation of age: 10.27

People earning <=50K:
Mean age: 36.61
Standard deviation of age: 13.46


# Task 5
Check, if there are some people without higher education (education: Bachelors, Prof-school, Assoc-acdm, Assoc-voc, Masters, Doctorate), but with > 50K salary

In [69]:
higher_education = [
    "Bachelors",
    "Prof-school",
    "Assoc-acdm",
    "Assoc-voc",
    "Masters",
    "Doctorate",
]

no_higher_education_but_high_income = df[
    ~df["education"].isin(higher_education) & (df["salary"] == ">50K")
]


print(
    f"There are {no_higher_education_but_high_income.shape[0]} people without higher education who earn >50K"
)

There are 3178 people without higher education who earn >50K


# Task 6
Get the statistics of age for each type of education. Use `groupby` and `describe` for this.

In [70]:
age_stats_by_education = df.groupby("education")["age"].describe()

print(age_stats_by_education)

               count       mean        std   min   25%   50%   75%   max
education                                                               
10th           820.0  37.897561  16.225795  17.0  23.0  36.0  52.0  90.0
11th          1048.0  32.363550  15.089307  17.0  18.0  28.5  43.0  90.0
12th           377.0  32.013263  14.373710  17.0  19.0  28.0  41.0  79.0
1st-4th        151.0  44.622517  14.929051  19.0  33.0  44.0  56.0  81.0
5th-6th        288.0  41.649306  14.754622  17.0  28.0  41.0  53.0  82.0
7th-8th        557.0  47.631957  15.737479  17.0  34.0  49.0  60.0  90.0
9th            455.0  40.303297  15.335754  17.0  28.0  38.0  53.0  90.0
Assoc-acdm    1008.0  37.286706  10.509755  19.0  29.0  36.0  44.0  90.0
Assoc-voc     1307.0  38.246366  11.181253  19.0  30.0  37.0  45.0  84.0
Bachelors     5044.0  38.641554  11.577566  19.0  29.0  37.0  46.0  90.0
Doctorate      375.0  47.130667  11.471727  24.0  39.0  47.0  54.0  80.0
HS-grad       9840.0  38.640955  13.067730  17.0  2

# Task 7
Compare the married and non-married men salaries. Who earns more? (>50K or <=50K)
Married men are those, whom `marital-status` starts with "Married". Others are not.

In [71]:
men_df = df[df["sex"] == "Male"]

married_men_df = men_df[men_df["marital-status"].str.startswith("Married")]
non_married_men_df = men_df[~men_df["marital-status"].str.startswith("Married")]

married_men_high_income = married_men_df[married_men_df["salary"] == ">50K"].shape[0]
married_men_low_income = married_men_df[married_men_df["salary"] == "<=50K"].shape[0]

non_married_men_high_income = non_married_men_df[
    non_married_men_df["salary"] == ">50K"
].shape[0]
non_married_men_low_income = non_married_men_df[
    non_married_men_df["salary"] == "<=50K"
].shape[0]

print("Married Men:")
print(f"Number earning >50K: {married_men_high_income}")
print(f"Number earning <=50K: {married_men_low_income}")

print("\nNon-Married Men:")
print(f"Number earning >50K: {non_married_men_high_income}")
print(f"Number earning <=50K: {non_married_men_low_income}")

if married_men_high_income > non_married_men_high_income:
    print("\nMarried men earn more than non-married men.")
elif married_men_high_income < non_married_men_high_income:
    print("\nNon-married men earn more than married men.")
else:
    print("\nMarried and non-married men earn the same amount (>50K).")

Married Men:
Number earning >50K: 5723
Number earning <=50K: 7052

Non-Married Men:
Number earning >50K: 673
Number earning <=50K: 6932

Married men earn more than non-married men.


# Task 8
Get the max hours per week some person works. How many people works the same amount of hours per week?

In [72]:
max_hours_per_week = df["hours-per-week"].max()

max_hours_count = df[df["hours-per-week"] == max_hours_per_week].shape[0]

print(f"Maximum hours per week: {max_hours_per_week}")
print(
    f"Number of people working {max_hours_per_week} hours per week: {max_hours_count}"
)

Maximum hours per week: 99
Number of people working 99 hours per week: 78


# Task 9
Analyze the correlation between data in dataset. Understand connected fields in it and print highlight thier connection.

In [80]:
df_numeric = df.apply(pd.to_numeric, errors="coerce")

df_numeric = df_numeric.dropna(axis=1, how="all")

correlation_matrix = df_numeric.corr()

print("Correlation Matrix:")
print(correlation_matrix)

Correlation Matrix:
                Unnamed: 0       age  hours-per-week  salary K$
Unnamed: 0        1.000000 -0.001126       -0.001890   0.000129
age              -0.001126  1.000000        0.101599   0.208203
hours-per-week   -0.001890  0.101599        1.000000   0.196378
salary K$         0.000129  0.208203        0.196378   1.000000
